In [20]:
from whatever.bk import DataSource, Figure, Join
from whatever.bk.fader import sweet_fade
from whatever.boilerplate.sklearn import load

import bokeh.io
import bokeh.models
import pandas
from toolz.curried import *

In [2]:
X, y, data = load('iris');
df = pandas.DataFrame(X, y)

In [3]:
import sklearn.discriminant_analysis
transformed = pandas.DataFrame(
    sklearn.discriminant_analysis
    .LinearDiscriminantAnalysis()
    .fit_transform(X, y),
    index=y, columns=['x', 'y']
)

In [15]:
props = {
    'text_alpha': {
        'lim': [.1 * transformed['x'].max(), 10*transformed['x'].min()],
        'value': [.9, .1],
    },
}

In [16]:
centers = transformed.groupby(df.index).mean()

In [22]:
with \
Figure(webgl=True, responsive=True) as figure, \
DataSource(transformed, x=0, y=1) as source, \
DataSource(centers, x=0, y=1) as center:
    source.add(
        transformed.index
        .map(['red', 'green', 'blue'].__getitem__),
        'fill_color'
    )
    Glyph = Join(figure, source)
    Glyph(
        bokeh.models.Circle, size=20, 
        fill_alpha=.6, line_alpha=0
    )

    center.add(
        centers.index.map(
            compose(str.upper, data['target_names'].__getitem__)
        ),
        'text'
    )

    sweet_fade(
        figure, Join(figure, center, bokeh.models.Text), props
    )
    
    bokeh.io.save(figure)